In [ ]:
!pip install av

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 43.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import pipeline
from transformers import VivitImageProcessor

image_processor = VivitImageProcessor.from_pretrained("std50218/vivit-b-16x2-kinetics400-0513-training-OM_M")

video_cls = pipeline(model="std50218/vivit-b-16x2-kinetics400-0513-training-OM_M")
video_cls.image_processor = image_processor

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/355M [00:00<?, ?B/s]

In [ ]:
import os
from collections import defaultdict
import numpy as np
import av
import torch
from transformers import VivitImageProcessor, VivitForVideoClassification
from tqdm import tqdm

def read_video_pyav(container, indices):
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices

def process_video_files(folder_paths):
    image_processor = video_cls.image_processor
    model = video_cls.model
    feature_dict = defaultdict(list)

    for folder_path in folder_paths:
        # 폴더명을 클래스 이름으로 사용
        for class_label in os.listdir(folder_path):
            class_path = os.path.join(folder_path, class_label)
            if not os.path.isdir(class_path):
                continue

            # 클래스 별 폴더 내 모든 파일을 탐색
            for filename in os.listdir(class_path):
                if filename.endswith(".mp4"):
                    file_path = os.path.join(class_path, filename)
                    container = av.open(file_path)

                    # 32 프레임 샘플링
                    indices = sample_frame_indices(clip_len=32, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
                    video = read_video_pyav(container=container, indices=indices)

                    # 비디오를 모델에 맞게 준비
                    video_array = np.array(video)
    inputs = image_processor(video_array, return_tensors="pt")
                    # 모델을 통한 전파
                    with torch.no_grad():
                        outputs = model(**inputs, output_hidden_states=True)
                        logits = outputs.logits
                        hidden_states = outputs.hidden_states
                        last_hidden = hidden_states[-1]

                    # 클래스별로 마지막 히든 레이어의 특징 저장
                    predictions = torch.argmax(logits, dim=-1)
                    for idx, prediction in enumerate(predictions):
                        feature_dict[class_label].append(last_hidden[idx].numpy())

    return feature_dict

# 폴더 경로 설정 및 함수 호출
folder_paths = ['/content/drive/MyDrive/기컴비_텀프/data/train_dataset/original_mediapipe/train', '/content/drive/MyDrive/기컴비_텀프/data/train_dataset/original_mediapipe/val',
                '/content/drive/MyDrive/기컴비_텀프/data/train_dataset/mediapipe/train', '/content/drive/MyDrive/기컴비_텀프/data/train_dataset/mediapipe/val']
all_features = process_video_files(folder_paths)
print(all_features)


TypeError: Cannot handle this data type: (1, 1, 1920, 3), |u1

In [ ]:
import pickle

with open("/content/drive/MyDrive/OM_M_all_features.pkl", 'wb') as f:
    pickle.dump(all_features, f)

In [ ]:
import numpy as np

def max_pooling_features(feature_dict):
    max_pooled_features = {}
    for class_label, features_list in feature_dict.items():
        features_array = np.array(features_list)
        max_pooled_feature = np.max(features_array, axis=0)
        max_pooled_features[class_label] = max_pooled_feature
    return max_pooled_features

# process_video_files()의 output: feature_dict
# max_pooling_features()의 input: feature_dict
max_pooled_features = max_pooling_features(all_features)
print(max_pooled_features)

{'가렵다': array([[ 7.4489155 , -0.6853926 , 12.664464  , ...,  3.1084394 ,
         6.9620094 ,  7.73977   ],
       [10.171316  ,  1.2113911 , 13.16649   , ...,  6.6404643 ,
         5.567769  , -2.890361  ],
       [10.430995  ,  4.306831  , 11.051609  , ...,  6.71488   ,
         5.3016095 , -4.5680823 ],
       ...,
       [10.478085  ,  0.5680922 , 11.575154  , ...,  8.930945  ,
         5.5147195 , -0.86358905],
       [10.901203  ,  0.6332119 , 11.8305025 , ...,  9.129835  ,
         5.408697  , -1.0855908 ],
       [11.140652  ,  1.1787745 , 12.404511  , ...,  8.993249  ,
         5.3254495 , -1.283715  ]], dtype=float32), '붕대': array([[11.994713 , -0.3375445,  8.456186 , ...,  3.5756226,  1.4361296,
        11.645455 ],
       [11.624998 ,  2.897863 , 13.07322  , ...,  4.201049 ,  4.31798  ,
        -3.0961204],
       [11.549082 ,  2.8045564, 12.969141 , ...,  4.48991  ,  4.3555393,
        -2.8959396],
       ...,
       [12.622358 ,  5.218207 , 11.701023 , ...,  6.2008

In [ ]:
with open("/content/drive/MyDrive/OM_M_max_pooled_features.pkl", 'wb') as f:
    pickle.dump(max_pooled_features, f)

In [ ]:
import numpy as np

def average_pooling_features(feature_dict):
    average_pooled_features = {}
    for class_label, features_list in feature_dict.items():
        features_array = np.array(features_list)
        average_pooled_feature = np.average(features_array, axis=0)
        average_pooled_features[class_label] = average_pooled_feature
    return average_pooled_features

# process_video_files()의 output: feature_dict
# max_pooling_features()의 input: feature_dict
average_pooled_features = average_pooling_features(all_features)
print(average_pooled_features)

{'가렵다': array([[  0.2578017 ,  -6.6420326 ,   3.8301575 , ...,  -4.601584  ,
          0.0724171 ,  -2.424583  ],
       [ -1.1352252 ,  -3.9139965 ,   3.6687348 , ...,  -1.9255565 ,
         -0.6376186 ,  -8.28478   ],
       [ -0.92087936,  -2.8922238 ,   2.4892335 , ...,  -2.8323398 ,
          0.1603895 , -12.518639  ],
       ...,
       [  0.0441809 ,  -4.248195  ,   2.71676   , ...,  -0.75983393,
         -0.697293  ,  -8.07978   ],
       [  0.22948575,  -4.1787057 ,   2.6587138 , ...,  -0.7057219 ,
         -0.5390642 ,  -8.319955  ],
       [  0.50054085,  -4.6180143 ,   3.0539844 , ...,  -0.7934152 ,
         -0.42621088,  -8.586859  ]], dtype=float32), '붕대': array([[ -1.2330394 ,  -7.0718527 ,   0.06484254, ...,  -3.619905  ,
         -3.1070776 ,  -0.4288713 ],
       [ -2.8476987 ,  -3.2385654 ,   1.836079  , ...,  -1.5858692 ,
         -2.0121245 ,  -8.451202  ],
       [ -2.4810271 ,  -2.3732016 ,   0.24243459, ...,  -2.2596946 ,
         -1.4657593 , -12.508635 

In [ ]:
with open("/content/drive/MyDrive/OM_M_average_pooled_features.pkl", 'wb') as f:
    pickle.dump(max_pooled_features, f)

In [ ]:
print(len(max_pooled_features))

10


In [ ]:
print(len(average_pooled_features))

10


In [ ]:
# import pickle

# # 파일로 저장
# with open("/content/drive/MyDrive/기컴비_텀프/code/feature_dictionary/OM_max_pooled_features.pkl", 'wb') as f:
#     pickle.dump(max_pooled_features, f)

# with open("/content/drive/MyDrive/기컴비_텀프/code/feature_dictionary/OM_average_pooled_features.pkl", 'wb') as f:
#     pickle.dump(max_pooled_features, f)

# with open("/content/drive/MyDrive/기컴비_텀프/code/feature_dictionary/OM_all_features.pkl", 'wb') as f:
#     pickle.dump(all_features, f)

# 파일 불러오기
# with open("/content/drive/MyDrive/max_pooled_features.pkl", 'rb') as f:
#     loaded_dict = pickle.load(f)